In [2]:
import tweepy
import time
import csv
import os
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.chdir('/content/drive/MyDrive')

In [ ]:
df = pd.read_parquet('final_twitter_data.parquet')
df.drop_duplicates(subset=['tweet_id'])

In [16]:
df.head(10)

,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type
0,2021-06-20T21:33:30.000Z,1406726930888540160,1406726930888540160,2981738470,RT @davidlohner: #IchbinHanna und schreibe auc...,7,0,0,0,"[{'type': 'retweeted', 'id': '1406714909258362...",['IchbinHanna'],retweet
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,Wenn Wirtschaftsjounalist:innen über #IchbinHa...,9,1,62,4,[],['IchbinHanna'],original
2,2021-06-20T20:50:42.000Z,1406716159064485895,1406716159064485895,2981738470,RT @DrKEichhorn: Wieder ein Artikel ohne echte...,16,0,0,0,"[{'type': 'retweeted', 'id': '1406697315159752...",[],retweet
3,2021-06-20T17:56:03.000Z,1406672208056012807,1406672208056012807,2981738470,RT @chrismeyer2203: So sieht es aus ⬇️ oder an...,7,0,0,0,"[{'type': 'retweeted', 'id': '1406175728190865...",[],retweet
4,2021-06-20T17:53:00.000Z,1406671439328055301,1406671439328055301,2981738470,RT @steffen_siegel: Modelle einer Universitäts...,17,0,0,0,"[{'type': 'retweeted', 'id': '1406666454368493...",[],retweet
5,2021-06-20T16:17:25.000Z,1406647386542325764,1406647386542325764,2981738470,☝️⬇️ #IchbinHanna #PeerReview https://t.co/PFN...,0,0,1,0,"[{'type': 'quoted', 'id': '1406620276822061057'}]","['IchbinHanna', 'PeerReview']",original
6,2021-06-20T15:17:28.000Z,1406632298183864333,1406632298183864333,2981738470,RT @Schaerferin: Mit 40 noch #IchbinHanna sein...,14,0,0,0,"[{'type': 'retweeted', 'id': '1406600563274493...",['IchbinHanna'],retweet
7,2021-06-20T15:16:35.000Z,1406632074908426241,1406632074908426241,2981738470,"RT @AmreiBahr: Hey @BMBF_Bund, nachdem Ihr let...",36,0,0,0,"[{'type': 'retweeted', 'id': '1406520362733654...",['IchBinHanna'],retweet
8,2021-06-19T09:49:55.000Z,1406187481662464001,1406187481662464001,2981738470,RT @jenniferhenkeHB: Einer habilitierten Kolle...,17,0,0,0,"[{'type': 'retweeted', 'id': '1406177602369462...",[],retweet
9,2021-06-18T20:27:59.000Z,1405985665154752513,1405985665154752513,2981738470,RT @SebastianKubon: #IchBinHanna die wunderbar...,7,0,0,0,"[{'type': 'retweeted', 'id': '1405960112884359...",['IchBinHanna'],retweet


In [27]:
list(df[df.tweet_id==1406632298183864333].hashtags)

["['IchbinHanna']"]

In [31]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util

In [40]:
model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')

In [85]:
def extract_most_frequent_hashtags(dataset):
    dates=list()

    for row,record in enumerate(df.hashtags):
      if record!='[]'and df['timestamp'][row][0:7]>'2021-05' : 
        dates.append(df['timestamp'][row][0:7])
    date=list()
    for dt in dates:
      if dt not in date:
        date.append(dt)




    i=1
    j=6
    for i in range(1,3):
      for j in range(1,13):
        if i==1 and j>5 and j<10 or i==2 and j<10:
          globals()[f'hash202{i}_0{j}']=list()
          globals()[f'hash202{i}_0{j}'].extend([hash for row,hash in enumerate(df['hashtags']) if df['timestamp'][row][0:7]==f'202{i}-0{j}' and hash!='[]'])
            
        elif i==1 and j>5 and j>=10 or i==2 and j>=10:
          globals()[f'hash202{i}_{j}']=list()
          globals()[f'hash202{i}_{j}'].extend([hash for row,hash in enumerate(df['hashtags']) if df['timestamp'][row][0:7]==f'202{i}-{j}' and hash!='[]'])
          
    #preprocessing the hashtags list from specific periods
    i=1
    j=6
    for i in range(1,3):
      for j in range(1,13):
        if i==1 and j>5 and j<10 or i==2 and j<10:
          globals()[f'all_hashtags_202{i}_0{j}']=list()
          for s in globals()[f'hash202{i}_0{j}']:
              bb=s.split(', ')
              a=[re.search(r"[\[\'\"]*(\w*)[\]\']*",i).group(1) for i in bb]
              globals()[f'all_hashtags_202{i}_0{j}'].extend(a)
            
            
        elif i==1 and j>5 and j>=10 or i==2 and j>=10:
          globals()[f'all_hashtags_202{i}_{j}']=list()
          for s in globals()[f'hash202{i}_{j}']:
              bb=s.split(', ')
              a=[re.search(r"[\[\'\"]*(\w*)[\]\']*",i).group(1) for i in bb]
              globals()[f'all_hashtags_202{i}_{j}'].extend(a)


    def hashtag_set_finder():
      hashtag_set=set()
      for dt in date:
          i,j = dt[3],dt[5:]
          lis = globals()[f'all_hashtags_202{i}_{j}']
          for i in lis:
            hashtag_set.add(i)
      return hashtag_set
    hashtags_in_DS=hashtag_set_finder()
    hashtags_in_DS2 = {i.lower() for i in hashtags_in_DS}
    def generate_hashtags(dataset):
      listOfAllHashtags=list()
      for row, hash in enumerate(dataset.hashtags):
        if dataset['timestamp'][row][0:10]>="2021-06-01" and hash!='[]':
          bb=hash.split(', ') 
          
          a=[re.search(r"[\[\']*(\w*)[\]\']*",i).group(1) for i in bb]
          listOfAllHashtags.extend(a)
      listOfAllHashtagsL=[i.lower() for i in listOfAllHashtags]
      return listOfAllHashtagsL
    list_of_all_hashtags = generate_hashtags(df)
    dic_of_mfh={i:list_of_all_hashtags.count(i)/len(list_of_all_hashtags) for i in hashtags_in_DS2 if list_of_all_hashtags.count(i)>500}
      #input_hashtags = [i[0] for i in list_of_mfh]
    return dic_of_mfh

In [86]:
def hashtag_groups():
    representative_hashtags=list()
    model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')
    dic_of_mfh= extract_most_frequent_hashtags(df)
    input_hashtags=list(dic_of_mfh.keys())
    sentence = input_hashtags
    embedding_full = model.encode(sentence)
    clusterd_hashtags=list()
    for hash in input_hashtags:
      embedding = model.encode(hash)
      sublist={hash}
      for j in range(len(input_hashtags)):
        if j<len(input_hashtags)-1:
          if util.pytorch_cos_sim(embedding,model.encode(input_hashtags[j+1])) > 0.5:
              sublist.add(input_hashtags[j+1])
              input_hashtags.remove(input_hashtags[j+1])
      clusterd_hashtags.append(sublist)

      clusterd_hashtags.sort(reverse=True, key= lambda i: len(i))
    print("for each cluster a group name has been extracted based on the most frequent hashtag of that group or a Group Representative\n")
    for i in clusterd_hashtags:
      i=list(i)
      group_n=i[0]
      for j in i:
        if dic_of_mfh[j] > dic_of_mfh[group_n]:
          group_n=j
      representative_hashtags.append(group_n)
      print(f"group name  :  {group_n}" )
      print(f'{i}\n') 
    return representative_hashtags

In [87]:
representative_hashtgas2=hashtag_groups()

for each cluster a group name has been extracted based on the most frequent hashtag of that group or a Group Representative

group name  :  ichbinhanna
['ichbinhannaat', 'ichbinhannach', 'ichbinhanna', 'ichbinreyhan', 'ichbinhannah']

group name  :  waspostdocswollen
['daad', 'waspostdocswollen', 'postdocs']

group name  :  hochschulen
['academia', 'hochschule', 'hochschulen']

group name  :  wisskomm
['woisthanna', 'wisskomm', 'ichwarhanna']

group name  :  dauerstellen
['dauerstellenfürdaueraufgaben', 'dauerstellen', 'befristung']

group name  :  hannaimbundestag
['hannastreikt', 'hannaimbundestag', 'hannainzahlen']

group name  :  academictwitter
['academicchatter', 'academictwitter']

group name  :  berlhg
['berlhg', 'hrk']

group name  :  wissenschaft
['forschung', 'wissenschaft']

group name  :  tvstud
['bundestag', 'tvstud']

group name  :  bmbf
['bmbf']

group name  :  27juni
['27juni']

group name  :  twittercampus
['twittercampus']

group name  :  dasgewinnenwir
['dasgewinnen

In [91]:
representative_hashtgas2

['ichbinhanna',
 'waspostdocswollen',
 'hochschulen',
 'wisskomm',
 'dauerstellen',
 'hannaimbundestag',
 'academictwitter',
 'berlhg',
 'wissenschaft',
 'tvstud',
 'bmbf',
 '27juni',
 'twittercampus',
 'dasgewinnenwir',
 'ugnovelle',
 'karliczek',
 'corona']

In [42]:
representative_hashtgas=hashtag_groups()

for each cluster a group name has been extracted based on the most frequent hashtag of that group or a Group Representative

group name  :  ichbinhanna
['ichbinhannaat', 'ichbinhannach', 'ichbinhanna', 'ichbinreyhan', 'ichbinhannah']

group name  :  waspostdocswollen
['daad', 'waspostdocswollen', 'postdocs']

group name  :  hochschulen
['academia', 'hochschule', 'hochschulen']

group name  :  wisskomm
['woisthanna', 'wisskomm', 'ichwarhanna']

group name  :  dauerstellen
['dauerstellenfürdaueraufgaben', 'dauerstellen', 'befristung']

group name  :  hannaimbundestag
['hannastreikt', 'hannaimbundestag', 'hannainzahlen']

group name  :  academictwitter
['academicchatter', 'academictwitter']

group name  :  berlhg
['berlhg', 'hrk']

group name  :  wissenschaft
['forschung', 'wissenschaft']

group name  :  tvstud
['bundestag', 'tvstud']

group name  :  bmbf
['bmbf']

group name  :  27juni
['27juni']

group name  :  twittercampus
['twittercampus']

group name  :  dasgewinnenwir
['dasgewinnen

In [43]:
representative_hashtgas

['ichbinhanna',
 'waspostdocswollen',
 'hochschulen',
 'wisskomm',
 'dauerstellen',
 'hannaimbundestag',
 'academictwitter',
 'berlhg',
 'wissenschaft',
 'tvstud',
 'bmbf',
 '27juni',
 'twittercampus',
 'dasgewinnenwir',
 'ugnovelle',
 'karliczek',
 'corona']

In [44]:
print(representative_hashtgas)

['ichbinhanna', 'waspostdocswollen', 'hochschulen', 'wisskomm', 'dauerstellen', 'hannaimbundestag', 'academictwitter', 'berlhg', 'wissenschaft', 'tvstud', 'bmbf', '27juni', 'twittercampus', 'dasgewinnenwir', 'ugnovelle', 'karliczek', 'corona']


In [81]:
def tweet_clustering(dataset,tweet_id,rep_hashtags):
  model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')
  hashclusters=set()
  tweet_hashtags=list()
  embedding_full = model.encode(rep_hashtags)
  hashtags=str(list(df[df.tweet_id==tweet_id].hashtags))
  hashtags=hashtags.split(', ')
  a=[re.search(r"[\[\'\"]*(\w*)[\]\']*",i).group(1) for i in hashtags]
  tweet_hashtags.extend(a)
  #print(a)
  if not tweet_hashtags[0]:
    print("no hashtags found for this tweet")
    return 

  for tw_hash in tweet_hashtags:
    embedding = model.encode(tw_hash)
    for rep_hash in rep_hashtags:
        if util.pytorch_cos_sim(embedding,model.encode(rep_hash)) > 0.5:
            hashclusters.add(rep_hash)

  return  hashclusters    

In [83]:
tweet_clustering(df,1406632298183864333,representative_hashtgas)

{'ichbinhanna'}

In [84]:
tweet_clustering(df,1406716159064485895,representative_hashtgas)

no hashtags found for this tweet
